In [1]:
"""
    This script loads and configures a processor in processors/, running it either locally or with TaskVine.
    The output parquet files are saved in results/{processor}/{year}/{type}.
    Run this script for each sample_json/NanoAODUL_{year}_{type}.json.
    To process specific datasets or root files, adjust the JSON files as needed.
    Use screen_run.py to automatically run this script through multiple JSONs.
    Raymond Kil, September 2025 (jkil@nd.edu)
"""
import os, glob, json, warnings
import pandas as pd
import importlib.util
from coffea import processor
from coffea.nanoevents import NanoAODSchema

In [2]:
### Directory Setup ###
#scriptPath = os.path.dirname(os.path.abspath(__file__))
outputPath = "data/results/makeDF/2018/mc/"
os.makedirs(outputPath, exist_ok=True)

In [3]:
### TaskVine Setup ###
#manager_name = f"{os.environ['USER']}-makeDF_2018_mc";
manager_name = os.environ.get("VINE_MANAGER_NAME")
print(manager_name)
ports_str = os.environ.get("VINE_MANAGER_PORTS", "9123, 9150")
ports = [int(p.strip()) for p in ports_str.split(",")]

floability-fe6a531f-e170-4566-9e75-53514a30cc0d


In [4]:
if len(ports) == 1:
    ports = ports[0]
else:
    ports = [int(p) for p in ports]

In [5]:
print(f"Manager Ports: {ports}")

Manager Ports: [9123, 9150]


In [6]:
if True:
    executor_args = {
        'desc'             : f'Processing makeDF',
        'manager_name'     : manager_name,
        'port'             : ports,
#        'environment_file' : f"{scriptPath}/lfv-coffea-env.tar.gz",
        'extra_input_files': glob.glob(f"utils/*"),
        'retries'          : 5,
        'fast_terminate_workers': 0
    }
    executor = processor.TaskVineExecutor(**executor_args)
else:
    executor_args = {
      'workers': 10,
      'desc': f'Processing makeDF'
    }
    executor = processor.FuturesExecutor(**executor_args)

In [7]:
print(f"nCores  : {int(os.cpu_count())}\nnWorkers: 10")

nCores  : 64
nWorkers: 10


In [8]:
samples={"tree":["data/samples/test/tree_1.root"]}

In [9]:
# module
processorpath = 'processors/makeDF.py' 
spec   = importlib.util.spec_from_file_location('my_processor', processorpath)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [10]:
# processor
warnings.filterwarnings("ignore", message=r"Missing cross-reference.*", module="coffea.nanoevents.schemas.nanoaod")
processor_instance = module.my_processor(year="2018", type="mc")
run = processor.Runner(
    executor      = executor,
    schema        = NanoAODSchema,
    maxchunks     = None,
    chunksize     = 20000,
    xrootdtimeout = 900,
    skipbadfiles  = False
)

In [11]:
### Running & Saving ###
result = run(samples, treename="Events", processor_instance=processor_instance)
for sampleName in samples.keys():
    dicts = {}
    for key, value in result.items():
        if sampleName==key[1]: dicts.update({key[0]:value})
    parqResult = pd.DataFrame([{key[0]:value for key,value in result.items() if sampleName in key[1]}])
    parqResult.to_parquet(f"{outputPath}/{sampleName}.parq")
    print(f"Results saved in {outputPath}/{sampleName}.parq")

GridspecLayout(children=(VBox(children=(HTML(value=''), HTML(value='')), layout=Layout(grid_area='widget001'))…

Output()

Listening for TaskVine workers on port 9123.

preprocessing task id 1 item pre_0 with 1 events on crcfe02.crc.nd.edu. return code -1 (input missing)

allocated cores: -1.0, memory: -1 MB, disk -1 MB, gpus: -1.0

...

allocated cores: -1.0, memory: -1 MB, disk -1 MB, gpus: -1.0

WARNING: task id 6 item pre_0 failed: input missing
    ('tree', 'data/samples/test/tree_1.root', 'Events') pre_0 with 'input missing' result.

WARNING: item pre_0 failed permanently.

2026/02/07 14:40:09.88 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory
2026/02/07 14:40:09.89 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory
2026/02/07 14:40:09.92 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory
2026/02/07 14:40:09.93 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory
2026/02/07 14:40:10.93 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory
2026/02/07 14:40:11.94 vine_manager[3110912]notice: input file data/samples/test/tree_1.root couldn't be accessed: No such file or directory


KeyboardInterrupt: 